# BGS Signal-to-Noise Ratio and Redshift Efficiency

The goal of this notebook is to assess the signal-to-noise ratio and redshift efficiency of BGS targets observed in "nominal" observing conditions (which are defined [here](https://github.com/desihub/desisurvey/blob/master/py/desisurvey/data/config.yaml#L102) and discussed [here](https://github.com/desihub/desisurvey/issues/77), among other places).  Specifically, the nominal BGS observing conditions we adopt (note the 5-minute exposure time is with the moon down!) are:

```python
{'AIRMASS': 1.0,
 'EXPTIME': 300,
 'SEEING': 1.1,
 'MOONALT': -60,
 'MOONFRAC': 0.0,
 'MOONSEP': 180}
```

During the survey itself, observations with the moon up (i.e., during bright time) will be obtained with longer exposure times according to the bright-time exposure-time model (see [here](https://github.com/desihub/surveysim/tree/master/doc/nb)).

Because we fix the observing conditions, we only consider how redshift efficiency depends on galaxy properties (apparent magnitude, redshift, 4000-A break, etc.).  However, note that the code is structured such that we *could* (now or in the future) explore variations in seeing, exposure time, and lunar parameters.

For code to generate large numbers of spectra over significant patches of sky and to create a representative DESI dataset (with parallelism), see `desitarget/bin/select_mock_targets` and `desitarget.mock.build.targets_truth`.

Finally, note that the various python Classes instantiated here (documented in `desitarget.mock.mockmaker`) are easily extensible to other mock catalogs and galaxy/QSO/stellar physics.



In [1]:
import os
import numpy as np
import matplotlib as mpl
mpl.use('Agg')
import matplotlib.pyplot as plt
from astropy.table import Table, vstack
from astropy.io import fits

/global/common/software/desi/cori/desiconda/20180130-1.2.4-spec/conda/lib/python3.6/site-packages/ipykernel/__main__.py:4: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/global/common/software/desi/cori/desiconda/20180130-1.2.4-spec/conda/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/global/common/software/desi/cori/desiconda/20180130-1.2.4-spec/conda/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/global/common/software/desi/cori/desiconda/20180130-1.2.4-spec/conda/lib/python3.6/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/global/common/software/desi/cori/desiconda/201

In [2]:
from desispec.io.util import write_bintable
from desiutil.log import get_logger, DEBUG
log = get_logger()
from desitarget.cuts import isBGS_bright, isBGS_faint
## Following not yet available in the master branch
from desitarget.mock.mockmaker import BGSMaker
from desitarget.mock.mockmaker import SKYMaker

In [3]:
import multiprocessing
nproc = multiprocessing.cpu_count() // 2 

import seaborn as sns
sns.set(style='white', font_scale=1.1, palette='deep')

In [4]:
# Specify if using this from command line as a .py or as an ipynb
using_py = False

class arg:
    pass

In [5]:
if using_py:
    import argparse
    parser = argparse.ArgumentParser()
    parser.add_argument('--sim', type=int, default=None, help='Simulation number (see documentation)')
    parser.add_argument('--part', type=str, default=None, help='Which part of the simulation to run. Options are all, newexp, group, zfit')
    args = parser.parse_args()
    if args.sim is None:
        parser.print_help()
        sys.exit(1)   
else:
    %matplotlib inline
    %load_ext autoreload
    %autoreload 2
    args = arg()
    args.sim = 1
    args.part = 'all'

#### Establish the I/O path, random seed, and path to the dust maps and desired healpixel.

In [6]:
simdir = os.path.join(os.getenv('DESI_ROOT'), 'spectro', 'sim', 'bgs', 'kremin', 'flat_priors')

if not os.path.exists(simdir):
    os.makedirs(simdir)

In [7]:
seed = 626

### All or none of the output files can be overwritten using these keywords.

In [8]:
overwrite_spectra = True
overwrite_templates = True
overwrite_redshifts = True
overwrite_results = True

#### Initialize random state

In [9]:
rand = np.random.RandomState(seed)

## Set up the simulation parameters.

Here we use the mock to capture the correct distribution of apparent magnitudes, galaxy properties, and redshifts.

Note that if `use_mock=False` then *rmagmin*, *rmagmax*, *zmin*, and *zmax* are required. For example, here's another possible simulation of 1000 spectra in which the magnitude (r=19.5) and redshift (z=0.2) are held fixed while moonfrac and moonsep are varied (as well as intrinsic galaxy properties):

```python
sim2 = dict(suffix='sim02',
            use_mock=False,
            nsim=10,
            nspec=100,
            seed=22,
            zmin=0.2, zmax=0.2,
            rmagmin=19.5, rmagmax=19.5,
            moonfracmin=0.0, moonfracmax=1.0,
            moonsepmin=0.0, moonsepmax=120.0,
           )
```

In [ ]:
from desistudy import get_predefined_sim_dict, get_predefined_obs_dict

all_sims = []
all_obsconds = []
#simnames = ['sim{:02d}'.format(args.sim)]
simnames = ['sim07','sim08']#['sim01','sim02','sim03','sim04']
for simname in simnames:
    all_sims.append(get_predefined_sim_dict(simname))
    all_obsconds.append(get_predefined_obs_dict(simname))

print(all_obsconds)

sims = np.atleast_1d(all_sims)
conditions = np.atleast_1d(all_obsconds)

[{'AIRMASS': 1.0, 'SEEING': 1.1, 'MOONALT': 30, 'MOONSEP': 120, 'exptimemin': 300, 'exptimemax': 720, 'MOONFRAC': 0.8}, {'AIRMASS': 1.0, 'SEEING': 1.1, 'MOONALT': 30, 'MOONSEP': 120, 'EXPTIME': 600, 'moonfracmin': 0.6, 'moonfracmax': 0.98}]


### Generate Spectra

In [ ]:
from desistudy import bgs_sim_spectra
from desistudy import bgs_redshifts
from desistudy import bgs_gather_results

for sim,cond in zip(sims,conditions):
    log.info("Now performing sim {}".format(sim['suffix']))
    bgs_sim_spectra(sim, cond, simdir, verbose=False, overwrite=overwrite_spectra)
    log.info("Finished simulating templates")
    bgs_redshifts(sim, simdir=simdir, overwrite=overwrite_redshifts)
    log.info("Finished redshift fitting")
    bgs_gather_results(sim, simdir=simdir, overwrite=overwrite_results)
    log.info("Finished gathering results")

INFO:<ipython-input-11-1eeb3b5cc42c>:6:<module>: Now performing sim sim07
INFO:io.py:1013:read_basis_templates: Reading /global/project/projectdirs/desi/spectro/templates/basis_templates/v2.5/bgs_templates_v2.1.fits metadata.
INFO:io.py:1025:read_basis_templates: Reading /global/project/projectdirs/desi/spectro/templates/basis_templates/v2.5/bgs_templates_v2.1.fits
Writing /global/project/projectdirs/desi/spectro/sim/bgs/kremin/flat_priors/sim07/bgs-sim07-simdata.fits
Writing /global/project/projectdirs/desi/spectro/sim/bgs/kremin/flat_priors/sim07/bgs-sim07-000-true.fits
INFO:quickspectra.py:54:sim_spectra: Starting simulation of 200 spectra


DEBUG:simexp.py:398:simulate_spectra: loading specsim desi config desi
DEBUG:simexp.py:402:simulate_spectra: creating specsim desi simulator


/global/common/software/desi/cori/desiconda/20180130-1.2.4-spec/conda/lib/python3.6/site-packages/astropy/table/column.py:1096: MaskedArrayFutureWarning: setting an item on a masked array which has a shared mask will not copy the mask and also change the original mask array in the future.
Check the NumPy 1.11 release notes for more information.
  ma.MaskedArray.__setitem__(self, index, value)


INFO:simexp.py:424:simulate_spectra: MJD not in obsconditions, using DATE-OBS 2009-06-18T12:00:00.000
DEBUG:simexp.py:428:simulate_spectra: obsconditions SEEING = 1.100000023841858
DEBUG:simexp.py:428:simulate_spectra: obsconditions EXPTIME = 355.096818357708
DEBUG:simexp.py:428:simulate_spectra: obsconditions AIRMASS = 1.0
DEBUG:simexp.py:428:simulate_spectra: obsconditions MOONFRAC = 0.800000011920929
DEBUG:simexp.py:428:simulate_spectra: obsconditions MOONALT = 30.0
DEBUG:simexp.py:428:simulate_spectra: obsconditions MOONSEP = 120.0
DEBUG:simexp.py:652:get_source_types: bgs 200 targets
DEBUG:simexp.py:468:simulate_spectra: running simulation with fastsim fiber loss method
DEBUG:simexp.py:474:simulate_spectra: source types: 200 bgs
INFO:quickspectra.py:203:sim_spectra: Wrote /global/project/projectdirs/desi/spectro/sim/bgs/kremin/flat_priors/sim07/bgs-sim07-000.fits
Writing /global/project/projectdirs/desi/spectro/sim/bgs/kremin/flat_priors/sim07/bgs-sim07-001-true.fits
INFO:quickspe

DEBUG:simexp.py:398:simulate_spectra: loading specsim desi config desi
DEBUG:simexp.py:402:simulate_spectra: creating specsim desi simulator
INFO:simexp.py:424:simulate_spectra: MJD not in obsconditions, using DATE-OBS 2009-06-18T12:00:00.000
DEBUG:simexp.py:428:simulate_spectra: obsconditions SEEING = 1.100000023841858
DEBUG:simexp.py:428:simulate_spectra: obsconditions EXPTIME = 568.6386975990126
DEBUG:simexp.py:428:simulate_spectra: obsconditions AIRMASS = 1.0
DEBUG:simexp.py:428:simulate_spectra: obsconditions MOONFRAC = 0.800000011920929
DEBUG:simexp.py:428:simulate_spectra: obsconditions MOONALT = 30.0
DEBUG:simexp.py:428:simulate_spectra: obsconditions MOONSEP = 120.0
DEBUG:simexp.py:652:get_source_types: bgs 200 targets
DEBUG:simexp.py:468:simulate_spectra: running simulation with fastsim fiber loss method
DEBUG:simexp.py:474:simulate_spectra: source types: 200 bgs
INFO:quickspectra.py:203:sim_spectra: Wrote /global/project/projectdirs/desi/spectro/sim/bgs/kremin/flat_priors/si

## Fit the redshifts.

This step took ~1.8 seconds per spectrum, ~3 minutes per 100 spectra, or ~30 minutes for all 1000 spectra with my 4-core laptop.

In [ ]:
# from desistudy import bgs_redshifts

# for sim in sims:
#     bgs_redshifts(sim, simdir=simdir, overwrite=overwrite_redshifts)

## Gather the results.

In [ ]:
# from desistudy import bgs_gather_results

# for sim in sims:
#     bgs_gather_results(sim, simdir=simdir, overwrite=overwrite_results)